In [139]:
import numpy as np
import pandas as pd
from ast import literal_eval
import re
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize

path = 'data/'

In [140]:
trial = pd.read_csv(path + 'tsd_trial.csv')
train = pd.read_csv(path + 'tsd_train.csv')

train['spans'] = train.spans.apply(literal_eval)
trial['spans'] = trial.spans.apply(literal_eval)

In [141]:
def getword(text, arr):
    ans = ''
    for i in range(len(arr)):
        elem = arr[i]
        if  i != 0 and i != len(arr) - 1 and elem != arr[i-1] + 1:
            ans += ' '
            ans += text[elem]
        else:
            ans += text[elem]
    return ans

In [142]:
def get_all_whitespaces(text):
    ans = []
    counter = 0
    for char in text:
        if char == ' ' or char == '\n':
            counter += 1
        elif counter != 0:
            ans.append(counter)
            counter = 0
    return ans


def cut_left_word_from_span(span):
    prev_i = 0
    for i in range(1, len(span)):
        if span[i] - span[prev_i] > 1:
            return span[i:]
        prev_i = i
    return []


def label_sentence(text, span_arr):
    whites = get_all_whitespaces(text)
    whites.append(1)
    sentence = text.split()
    new_sentence = []
    curr_position = 0
    in_span = False
    for i, word in enumerate(sentence):
        if len(span_arr) != 0:
            if not in_span and curr_position == span_arr[0]:
                in_span = True
                new_word = (word, 'I-SPAN')
            elif in_span:
                if curr_position in span_arr:
                    new_word = (word, 'I-SPAN')
                else:
                    in_span = False
                    span_arr = cut_left_word_from_span(span_arr)
                    new_word = (word, '0')
            else:
                new_word = (word, 'O')
            curr_position += len(word) + whites[i]
        else:
            new_word = (word, 'O')
        new_sentence.append(new_word)
    return new_sentence

In [143]:
def conll_sentence(labeled_sentence):
    ok_sentence = []
    for elem in labeled_sentence:
        new_word = []
        word, label = elem
        word_elems = word_tokenize(word)
        for word_elem in word_elems:
            if any(char.isalpha() for char in word_elem):
                new_word.append((word_elem, label))
            else:
                new_word.append((word_elem, 'O'))
        ok_sentence += new_word
    return ok_sentence

In [144]:
text = train.iloc[8, 1]
span = train.iloc[8, 0]
print(text, span, sep = "\n")
print(getword(text, span))
test = label_sentence(text, span)
print(test)

Obamacare is on it's last gasping breaths.   You idiots who don't want something else passed that's going to save your free healthcare are kind of stupid.   But if you block anything else that gives you socialized healthcare it's your own fault for being unbudging.  You can go back to nothing when nothing further is passed and Obamacare is just an ugly memory.
[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 151, 152]
idiots stupid
[('Obamacare', 'O'), ('is', 'O'), ('on', 'O'), ("it's", 'O'), ('last', 'O'), ('gasping', 'O'), ('breaths.', 'O'), ('You', 'O'), ('idiots', 'I-SPAN'), ('who', '0'), ("don't", 'O'), ('want', 'O'), ('something', 'O'), ('else', 'O'), ('passed', 'O'), ("that's", 'O'), ('going', 'O'), ('to', 'O'), ('save', 'O'), ('your', 'O'), ('free', 'O'), ('healthcare', 'O'), ('are', 'O'), ('kind', 'O'), ('of', 'O'), ('stupid.', 'I-SPAN'), ('But', '0'), ('if', 'O'), ('you', 'O'), ('block', 'O'), ('anything', 'O'), ('else', 'O'), ('that', 'O'), ('gives', 'O'), ('you', 'O'), ('social

In [145]:
train_raw_conll = []
for i, text, span in zip(range(len(train)), train['text'], train['spans']):
    try:
        train_raw_conll.append(label_sentence(text, span))
    except:
        print(i)


In [146]:
train_conll = []
for sent in train_raw_conll:
    try:
        train_conll.append(conll_sentence(sent))
    except:
        print(i)

In [147]:
print(train_conll[125])

[('I', 'O'), ('suppose', 'O'), ('the', 'O'), ('Russians', 'I-SPAN'), ('were', '0'), ('punished', 'O'), ('for', 'O'), ('Stalin', 'O'), ("'s", 'O'), ('misdeeds', 'O'), ('?', 'O'), ('Gosh', 'O'), ('you', 'O'), ('are', 'O'), ('a', 'O'), ('stupid', 'I-SPAN'), ('Dork', '0'), ('!', 'O')]


In [148]:
trial_raw_conll = []
for i, text, span in zip(range(len(trial)), trial['text'], trial['spans']):
    try:
        trial_raw_conll.append(label_sentence(text, span))
    except:
        print(i)

In [149]:
trial_conll = []
for sent in trial_raw_conll:
    try:
        trial_conll.append(conll_sentence(sent))
    except:
        print(i)

In [150]:
print(len(trial_conll))

690


In [154]:
with open('./data/trial.conll', 'w') as f:
    for sentence in trial_conll:
        f.write('-DOCSTART- O\n')
        f.write('\n')
        for comb in sentence:
            f.write(comb[0] + ' ' + comb[1] + '\n')
        f.write('\n')
    

In [155]:
with open('./data/train.conll', 'w') as f:
    for sentence in train_conll:
        f.write('-DOCSTART- O\n')
        f.write('\n')
        for comb in sentence:
            f.write(comb[0] + ' ' + comb[1] + '\n')
        f.write('\n')
    

In [156]:
print(len(train))

7939
